# Email Service - Send to all customers

### Download dependencies

In [ ]:
%pip install openai 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 21.1 MB/s eta 0:00:00:00:0100:01


### Load Zero Shot prompt

In [ ]:
with open("prompts/political_text_classifer.txt", "r", encoding="utf-8") as file:
    instructions = file.read()

### Define function to run the political text classifer

In [ ]:
from openai import OpenAI
client = OpenAI()

def classifyText(input_text: str):
    response = client.chat.completions.create(
        model="gpt-4o",
        instructions=instructions,
        input=input_text
    )

    return response

### Run webscraper to get NYC legislation